In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np
import os
import re
import lxml

In [2]:
driver = webdriver.Chrome("C:\\Users\\bobby\\Desktop\\Python_project\\chromedriver.exe")

In [91]:
#load up the athlete page and accept the cookies tab
#driver.get("https://www.ufc.com/athletes/all")
#ele=driver.find_element_by_id("onetrust-accept-btn-handler")
#time.sleep(1)
#ele.click()
#time.sleep(2)

In [97]:
#to find the total of fighter
#total=BeautifulSoup(driver.page_source,"lxml")
#totalAth=total.find_all("div", {"class": "althelete-total"})
#stotal=re.search("\d+",str(totalAth)).group(0)
#stotal=int(int(stotal)/11)


In [98]:
#load up the fighter pages
#for i in range(stotal):
#    ele=driver.find_element_by_class_name("pager__item")
#    ele.click()
#    time.sleep(5)

In [99]:
#put the links into a data frame and out a csv
#total=BeautifulSoup(driver.page_source,"lxml")
#links=total.find_all("a", {"class": "e-button--black"})
#ufc = "https://www.ufc.com"
#links[0]["href"]
#linksAH=pd.DataFrame(np.nan, index=range(len(links)), columns=["link"],dtype="string")
#for i in range(len(links)):
#        linksAH.at[i,'link']= ufc +str(links[i]["href"])
#linksAH.to_csv("links_to_fighters.csv",index =False)

In [111]:
pd.read_csv("links_to_fighters.csv")

,link
0,https://www.ufc.com/athlete/shamil-abdurakhimov
1,https://www.ufc.com/athlete/ricardo-abreu
2,https://www.ufc.com/athlete/klidson-abreu
3,https://www.ufc.com/athlete/juan-adams
4,https://www.ufc.com/athlete/zarrukh-adashev
...,...
1113,https://www.ufc.com/athlete/lv-zhenhong
1114,https://www.ufc.com/athlete/zhalgas-zhumagulov
1115,https://www.ufc.com/athlete/fares-ziam
1116,https://www.ufc.com/athlete/cat-zingano


In [16]:
driver.get(linksAH.iloc[0,0])
time.sleep(2)

In [17]:
ele = driver.find_element_by_class_name("e-button--white")
ele.click()

while True:
    try:
        ele = driver.find_element_by_class_name("pager__item")
        ele.click()
        time.sleep(2)
    except:
        break

In [73]:
time.sleep(2)
stats = BeautifulSoup(driver.page_source,"lxml")
columns = ["name","str.ac","grap.ac","sig.str.land","sig.str.att","take.land","take.att",
           "sig.str.land.min","sig.str.abs.min","take.avg.15min","sub.avg.15min","sig.str.def",
          "take.def","knock.ratio","avg.time","sig.str.head","sig.str.head.percent","sig.str.body",
          "sig.str.body.percent","sig.str.leg","sig.str.leg.percent",
          "sig.str.position.stand","sig.str.position.clinch","sig.str.position.ground",
          "sig.str.position.stand.percent","sig.str.position.clinch.percent","sig.str.position.ground.percent",
           "win.ko","win.sub","win.dec","win.ko.percent","win.sub.percent","win.dec.percent",
          "age","leg.reach","reach","height","opponent","Win"]
StatsOfFigthers=pd.DataFrame(columns =columns)
temp = pd.DataFrame(columns =columns,index =range(1))

In [74]:
#find striking acc and grappling acc
for a in stats.find_all(attrs ={"class":"c-overlap-athlete-detail__card"}):
    title =re.sub("<.*?>","",str(a.title))
    title =re.sub("\d+%","",title)
    title = title.strip()
    title = title.lower()
    if title == "striking accuracy":
        temp["str.ac"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"})))
        temp["sig.str.land"] = re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0]))
        temp["sig.str.att"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1]))
    if title == "grappling accuracy":
        temp["grap.ac"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"})))
        temp["take.land"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0]))
        temp["take.att"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1]))


In [75]:

for a in stats.find_all(attrs={"class":"c-stat-compare__group-1"}):
    label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
    num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))
    label=label.strip()
    label=label.lower()
    if label == "sig. str. landed":
        temp["sig.str.land.min"]=num
        
    if label == "takedown avg":
        temp["take.avg.15min"]=num
        
    if label == "sig. str. defense":
        temp["sig.str.def"]= num
        
    if label == "knockdown ratio":
        temp["knock.ratio"]= num
        
for a in stats.find_all(attrs={"class":"c-stat-compare__group-2"}):
    label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
    num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))
    label=label.strip()
    label=label.lower()
    if label == "sig. str. absorbed":
        temp["sig.str.abs.min"]=num
        
    if label == "submission avg":
        temp["sub.avg.15min"]=num
        
    if label == "takedown defense":
        temp["take.def"]= num
        
    if label == "average fight time":
        temp["avg.time"]= num



In [76]:
temp["sig.str.head"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_value"})))
temp["sig.str.head.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_percent"})))
temp["sig.str.body"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_value"})))
temp["sig.str.body.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_percent"})))
temp["sig.str.leg"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_value"})))
temp["sig.str.leg.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_percent"})))



In [77]:
for a in stats.find_all(attrs={"class":"c-bio__field"}):
    lab=a.find(attrs={"class":"c-bio__label"})
    lab = re.sub("<.*?>","",str(lab))
    c=re.sub("<.*?>","",str(a))
    c= re.search("\d+",str(c))
    lab=lab.strip()
    lab = lab.lower()
    if(lab=="age"):
        temp["age"]=c.group(0)
    if(lab=="height"):
        temp["height"]=c.group(0)
    if(lab=="reach"):
        temp["reach"]=c.group(0)
    if(lab=="leg reach"):
        temp["leg.reach"]=c.group(0)

In [78]:
for a in stats.find_all(attrs={"class":"c-stat-3bar__group"}):
    label =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-3bar__label"})))
    num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-3bar__value"})))
    label=label.lower()
    label = label.strip()
    if label == "standing":
        temp["sig.str.position.stand"]=re.sub("\\(.+\\)","",num)
        temp["sig.str.position.stand.percent"]=re.findall("\\(.+\\)",num)
    if label == "clinch":
        temp["sig.str.position.clinch"]=re.sub("\\(.+\\)","",num)
        temp["sig.str.position.clinch.percent"]=re.findall("\\(.+\\)",num)
    if label == "ground":
        temp["sig.str.position.ground"]=re.sub("\\(.+\\)","",num)
        temp["sig.str.position.ground.percent"]=re.findall("\\(.+\\)",num)
    if label == "ko/tko":
        temp["win.ko"] = re.sub("\\(.+\\)","",num)
        temp["win.ko.percent"] = re.findall("\\(.+\\)",num)
    if label == "dec":
        temp["win.dec"] = re.sub("\\(.+\\)","",num)
        temp["win.dec.percent"] = re.findall("\\(.+\\)",num)
    if label == "sub":
        temp["win.sub"] = re.sub("\\(.+\\)","",num)
        temp["win.sub.percent"] = re.findall("\\(.+\\)",num)

In [79]:
listofights =[]
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__matchup"}):
    for j in a:
        listofights.append(j)
    

In [80]:
for i in range(len(listofights)):
    for a in range(len(listofights)):
        try:
            if str(listofights[a]) == "\n":
                listofights.pop(a)
        except:
            break


In [81]:
listoffightnames =[]
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__headline"}):
    listoffightnames.append(re.sub("<.*?>","",str(a)))

In [82]:
name=re.search("-.*",str(linksAH.iloc[0,0])).group(0)
name=re.sub("-","",name)

In [83]:
name=name.strip()
name=name.lower()

In [84]:
temp["name"] = name

In [85]:
temp=pd.concat([temp] *len(listoffightnames),ignore_index=True)

In [86]:
for i in range(0,len(listofights),2):
    
    currentnames=listoffightnames[int(i/2)]
    currentnames=re.sub("vs","",currentnames)
    leftname=re.search(".*? +",currentnames).group(0)
    rightname=re.search(" +.*",currentnames).group(0)
    leftname=leftname.strip()
    leftname=leftname.lower()
    rightname=rightname.strip()
    rightname=rightname.lower()
    
    if leftname == name:
        temp.iloc[int(i/2)]["opponent"] = rightname
        if re.sub("<.*?>","",str(listofights[i])).strip() != "Win" and re.sub("<.*?>","",str(listofights[i+1])).strip() != "Win":
            temp.iloc[int(i/2)]["Win"] = "Draw"
        else:
            temp.iloc[int(i/2)]["Win"]=re.sub("<.*?>","",str(listofights[i])).strip()
    else:
        temp.iloc[int(i/2)]["opponent"] = leftname
        if re.sub("<.*?>","",str(listofights[i])).strip() != "Win" and re.sub("<.*?>","",str(listofights[i+1])).strip() != "Win":
            temp.iloc[int(i/2)]["Win"] = "Draw"
        else:
            temp.iloc[int(i/2)]["Win"]=re.sub("<.*?>","",str(listofights[i+1])).strip()

In [87]:
temp

,name,str.ac,grap.ac,sig.str.land,sig.str.att,take.land,take.att,sig.str.land.min,sig.str.abs.min,take.avg.15min,...,win.dec,win.ko.percent,win.sub.percent,win.dec.percent,age,leg.reach,reach,height,opponent,Win
0,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,daukaus,Draw
1,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,blaydes,
2,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,tybura,Win
3,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,arloki,Win
4,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,sherman,Win
5,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,lewis,
6,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,harris,Win
7,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,hamilton,Win
8,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,johnson,


In [90]:
StatsOfFigthers=StatsOfFigthers.append(temp)

,name,str.ac,grap.ac,sig.str.land,sig.str.att,take.land,take.att,sig.str.land.min,sig.str.abs.min,take.avg.15min,...,win.dec,win.ko.percent,win.sub.percent,win.dec.percent,age,leg.reach,reach,height,opponent,Win
0,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,daukaus,Draw
1,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,blaydes,
2,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,tybura,Win
3,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,arloki,Win
4,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,sherman,Win
5,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,lewis,
6,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,harris,Win
7,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,hamilton,Win
8,abdurakhimov,[45%],[24%],210,469,5,29,2.45\n,2.45\n,1.23\n,...,7,(45%),(20%),(35%),39,41,76,75,johnson,


In [103]:
linksAH

,link
0,https://www.ufc.com/athlete/shamil-abdurakhimov
1,https://www.ufc.com/athlete/ricardo-abreu
2,https://www.ufc.com/athlete/klidson-abreu
3,https://www.ufc.com/athlete/juan-adams
4,https://www.ufc.com/athlete/zarrukh-adashev
...,...
1113,https://www.ufc.com/athlete/lv-zhenhong
1114,https://www.ufc.com/athlete/zhalgas-zhumagulov
1115,https://www.ufc.com/athlete/fares-ziam
1116,https://www.ufc.com/athlete/cat-zingano
